In [2]:
# Import libraries
import numpy as np
import pandas as pd
from fastai.tabular.all import TabularDataLoaders, Normalize, tabular_learner, R2Score, rmse, Categorify

In [3]:
# dictionary: block and street to lat/long
latLong = {
  '212 Marsiling Cres': [1.44622, 103.77311],
  '1 Marsiling Dr': [1.44067, 103.77571],
  '2 Marsiling Dr': [1.43995, 103.77655],
  '6 Marsiling Dr': [1.44208, 103.77633],
  '22 Marsiling Dr': [1.44191, 103.77436],
  '23 Marsiling Dr': [1.44151, 103.77330],
  '24 Marsiling Dr': [1.44154, 103.77210],
  '30 Marsiling Dr': [1.44258, 103.77240],
  '31 Marsiling Dr': [1.44305, 103.77232],
  '32 Marsiling Dr': [1.44337, 103.77293],
  '202 Marsiling Dr': [1.44289, 103.77487],
  '204 Marsiling Dr': [1.44408, 103.77370],
  '206 Marsiling Dr': [1.44537, 103.77293],
  '12 Marsiling Lane': [1.44061, 103.77870],
  '12A Marsiling Lane': [1.44141, 103.77849],
  '13 Marsiling Lane': [1.44109, 103.77925],
  '15 Marsiling Lane': [1.44198, 103.77847],
  '17 Marsiling Lane': [1.44273, 103.77804],
  '18 Marsiling Lane': [1.44342, 103.77787],
  '19 Marsiling Lane': [1.44328, 103.77730],
  '180A Marsiling Rd': [1.43913, 103.77814],
  '180B Marsiling Rd': [1.43913, 103.77835],
  '125 Marsiling Rise': [1.43849, 103.77953],
  '129 Marsiling Rise': [1.43783, 103.77871],
  '130 Marsiling Rise': [1.43812, 103.77870],
  '131 Marsiling Rise': [1.43849, 103.77862],
  '132 Marsiling Rise': [1.43829, 103.77803],
  '570A Woodlands Ave 1': [1.43071, 103.79105],
  '570B Woodlands Ave 1': [1.43091, 103.79077],
  '571C Woodlands Ave 1': [1.43040, 103.78997],
  '693A Woodlands Ave 6': [1.43864, 103.80377],
  '693B Woodlands Ave 6': [1.43834, 103.80388],
  '693C Woodlands Ave 6': [1.43793, 103.80407],
  '780A Woodlands Cres': [1.44882, 103.80067],
  '780B Woodlands Cres': [1.44902, 103.80102],
  '780C Woodlands Cres': [1.44913, 103.80139],
  '780F Woodlands Cres': [1.44788, 103.80124],
  '782C Woodlands Cres': [1.44780, 103.80344],
  '782D Woodlands Cres': [1.44823, 103.80283],
  '573A Woodlands Dr 16': [1.43133, 103.79828],
  '573B Woodlands Dr 16': [1.43094, 103.79781],
  '573C Woodlands Dr 16': [1.43052, 103.79767],
  '574A Woodlands Dr 16': [1.43011, 103.79710],
  '574B Woodlands Dr 16': [1.42970, 103.79685],
  '886A Woodlands Dr 50': [1.43830, 103.79489],
  '886B Woodlands Dr 50': [1.43824, 103.79387],
  '886D Woodlands Dr 50': [1.43794, 103.79490],
  '887B Woodlands Dr 50': [1.43710, 103.79406],
  '887C Woodlands Dr 50': [1.43697, 103.79468],
  '694A Woodlands Dr 62': [1.43916, 103.80419],
  '694B Woodlands Dr 62': [1.43926, 103.80517],
  '694C Woodlands Dr 62': [1.43862, 103.80561],
  '694D Woodlands Dr 62': [1.43843, 103.80544],
  '717A Woodlands Dr 70': [1.44049, 103.79836],
  '717B Woodlands Dr 70': [1.44135, 103.79861],
  '784B Woodlands Rise': [1.44565, 103.80499],
  '784C Woodlands Rise': [1.44602, 103.80500],
  '785B Woodlands Rise': [1.44636, 103.80527],
  '101 Woodlands St 13': [1.43612, 103.78206],
  '102 Woodlands St 13': [1.43611, 103.78117],
  '111 Woodlands St 13': [1.43784, 103.78219],
  '145 Woodlands St 13': [1.43612, 103.77324],
  '146 Woodlands St 13': [1.43612, 103.77409],
  '148 Woodlands St 13': [1.43578, 103.77402],
  '149 Woodlands St 13': [1.43578, 103.77303],
  '151 Woodlands St 13': [1.43546, 103.77423],
  '152 Woodlands St 13': [1.43505, 103.77424],
  '155 Woodlands St 13': [1.43477, 103.77459],
  '157 Woodlands St 13': [1.43437, 103.77480],
  '158 Woodlands St 13': [1.43407, 103.77478],
  '159 Woodlands St 13': [1.43371, 103.77459],
  '160 Woodlands St 13': [1.43337, 103.77469],
  '161 Woodlands St 13': [1.43337, 103.77367],
  '166 Woodlands St 13': [1.43374, 103.77543],
  '302A Woodlands St 31': [1.43185, 103.77527],
  '310 Woodlands St 31': [1.43009, 103.77598],
  '311 Woodlands St 31': [1.43185, 103.77527],
  '313 Woodlands St 31': [1.43049, 103.77617],
  '314 Woodlands St 31': [1.43119, 103.77607],
  '316 Woodlands St 31': [1.43149, 103.77637],
  '317 Woodlands St 31': [1.43190, 103.77627],
  '318 Woodlands St 31': [1.43185, 103.77731],
  '319 Woodlands St 31': [1.43150, 103.77740],
  '411 Woodlands St 41': [1.42852, 103.77282],
  '419 Woodlands St 41': [1.42852, 103.77457],
  '818 Woodlands St 82': [1.44167, 103.79018],
  '820 Woodlands St 82': [1.44101, 103.79014],
  '848 Woodlands St 82': [1.44181, 103.79198],
  '849 Woodlands St 82': [1.44147, 103.79199],
  '834 Woodlands St 83': [1.44009, 103.79108],
}

In [4]:
# load data frame

def load_csv():
  return pd.read_csv('./data/2305_3rm_woodlands.csv')
df = load_csv()
df

,block,street,floor,area,year,lease,price,date
0,24,Marsiling Dr,01 to 03,67.00\nNew Generation,1977,53 years\n2 months,"$300,500.00",05/01/23
1,15,Marsiling Lane,10 to 12,65.00\nImproved,1976,51 years\n9 months,"$315,000.00",05/01/23
2,180A,Marsiling Rd,22 to 24,67.00\nModel A,2015,91 years\n3 months,"$430,000.00",05/01/23
3,693B,Woodlands Ave 6,07 to 09,68.00\nModel A,2018,94 years\n5 months,"$445,000.00",05/01/23
4,694C,Woodlands Dr 62,07 to 09,68.00\nModel A,2018,94 years\n5 months,"$450,000.00",05/01/23
...,...,...,...,...,...,...,...,...
302,782C,Woodlands Cres,07 to 09,67.00\nModel A,2015,92 years\n2 months,"$395,000.00",05/01/22
303,886A,Woodlands Dr 50,13 to 15,67.00\nPremium Apartment,2015,91 years\n9 months,"$420,000.00",05/01/22
304,717A,Woodlands Dr 70,04 to 06,71.00\nModel A,2017,94 years\n8 months,"$420,000.00",05/01/22
305,785B,Woodlands Rise,07 to 09,68.00\nModel A,2018,95 years\n3 months,"$400,000.00",05/01/22


In [5]:
# clean up price (currency string to float)

# inplace cleanup
def price_cleanup(df):
  df['price'] = df.price.apply(lambda x:float(x.replace('$','').replace(',','')) if isinstance(x,str) else x)
  return df
price_cleanup(df)
df


,block,street,floor,area,year,lease,price,date
0,24,Marsiling Dr,01 to 03,67.00\nNew Generation,1977,53 years\n2 months,300500.0,05/01/23
1,15,Marsiling Lane,10 to 12,65.00\nImproved,1976,51 years\n9 months,315000.0,05/01/23
2,180A,Marsiling Rd,22 to 24,67.00\nModel A,2015,91 years\n3 months,430000.0,05/01/23
3,693B,Woodlands Ave 6,07 to 09,68.00\nModel A,2018,94 years\n5 months,445000.0,05/01/23
4,694C,Woodlands Dr 62,07 to 09,68.00\nModel A,2018,94 years\n5 months,450000.0,05/01/23
...,...,...,...,...,...,...,...,...
302,782C,Woodlands Cres,07 to 09,67.00\nModel A,2015,92 years\n2 months,395000.0,05/01/22
303,886A,Woodlands Dr 50,13 to 15,67.00\nPremium Apartment,2015,91 years\n9 months,420000.0,05/01/22
304,717A,Woodlands Dr 70,04 to 06,71.00\nModel A,2017,94 years\n8 months,420000.0,05/01/22
305,785B,Woodlands Rise,07 to 09,68.00\nModel A,2018,95 years\n3 months,400000.0,05/01/22


In [6]:
# clean up area, date, floor
df = load_csv()
price_cleanup(df)

def area_cleanup(df):
  df['area'] = df.area.str.split('\n',expand=True)[0].astype(float) if df.area.dtype != 'float64' else df.area
  return df  
area_cleanup(df)

def date_cleanup(df):
  df['date'] = pd.to_datetime(df.date)
  df['date'] = df.date.apply(lambda x:float(x.toordinal()) if not isinstance(x,float) else x)
  return df
date_cleanup(df)

floor_dict = { f'0{i*3+1} to 0{i*3+3}' if i < 3 else f'{i*3+1} to {i*3+3}': i*3+2  for i in range(11)  }
def floor_cleanup(df):
  df['floor'] = df.floor.apply(lambda x:float(floor_dict[x]) if isinstance(x,str) else x)
  return df
floor_cleanup(df)

df.describe()



/tmp/ipykernel_17920/2230633797.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df.date)


,floor,area,year,price,date
count,307.000000,307.000000,307.000000,307.000000,307.000000
mean,7.609121,68.589577,1997.941368,372572.586319,738459.328990
std,5.095524,3.940027,17.531200,53452.277518,102.674016
min,2.000000,60.000000,1974.000000,248888.000000,738276.000000
25%,5.000000,67.000000,1981.000000,324500.000000,738368.000000
50%,8.000000,68.000000,1993.000000,375000.000000,738460.000000
75%,11.000000,68.000000,2018.000000,418444.000000,738552.000000
max,32.000000,88.000000,2018.000000,490000.000000,738641.000000


In [27]:
# Learner
dls = TabularDataLoaders.from_df(df,
                                 y_names='price',
                                  cont_names = ['year', 'floor', 'area', 'date'],
                                  procs = [Normalize],
                                  valid_idx=range(60))
dls.show_batch()

,year,floor,area,date,price
0,1995.000000,2.0,69.0,738367.999998,335000.0
1,2018.000000,2.0,68.0,738551.999997,425000.0
2,2018.000000,2.0,68.0,738429.000000,410000.0
3,1981.000000,8.0,68.0,738521.000005,320000.0
4,1983.000000,5.0,74.0,738276.000001,265000.0
5,1976.000000,5.0,65.0,738307.000003,277000.0
6,2018.000000,5.0,68.0,738490.000002,428000.0
7,2013.000000,11.0,67.0,738490.000002,440000.0
8,2018.000000,2.0,68.0,738460.000000,400000.0
9,1977.000001,5.0,67.0,738399.000000,315000.0


In [42]:
# Naive attempt: categorify everything else
dls = TabularDataLoaders.from_df(train_set,
                                  y_names='price',
                                  cat_names = ['floor', 'area', 'date'],
                                  cont_names = ['year'],
                                  procs = [Normalize, Categorify],
                                  valid_idx=[i for i in range(0,60)]
)
dls.show_batch()

,floor,area,date,year,price
0,04 to 06,74.00\nModel A,05/01/22,1983.000000,265000.0
1,07 to 09,73.00\nModel A,09/01/22,1990.000000,380000.0
2,04 to 06,68.00\nNew Generation,06/01/22,1984.999999,350000.0
3,07 to 09,68.00\nModel A,11/01/22,2018.000000,420000.0
4,04 to 06,73.00\nModel A,11/01/22,1984.999999,352000.0
5,01 to 03,68.00\nModel A,08/01/22,2018.000000,415000.0
6,07 to 09,82.00\nNew Generation,07/01/22,1976.999999,350000.0
7,10 to 12,68.00\nModel A,12/01/22,2018.000000,462000.0
8,01 to 03,65.00\nImproved,09/01/22,1975.999999,280000.0
9,04 to 06,73.00\nModel A,05/01/22,1990.000000,358000.0


In [135]:
# get median house
medianPrice = df.price.quantile(interpolation='nearest')
medianLoc = df.price[df.price==medianPrice].idxmin()
print(medianLoc)
print(df.iloc[medianLoc])

25
block                     311
street        Woodlands St 31
floor                07 to 09
area           73.00\nModel A
top                      1993
lease     69 years\n6 months 
price                375000.0
date                 04/01/23
Name: 25, dtype: object


In [136]:
# seeded validation set
valid_idx = np.random.default_rng(seed=42).integers(0,len(df),size=60)
if medianLoc not in valid_idx: valid_idx = np.append(valid_idx, medianLoc)
print(valid_idx)


[ 27 237 200 134 132 263  26 214  61  28 161 299 225 233 220 241 157  39
 257 138 153 113  56 284 239 197 123 252 167 136 138  69  28 170 272  19
 263 254  84 193  50 232 215 108  20 298 136 274 208 238 233  59 111 143
 152  13 167  47 228 209  25]


In [184]:
# master clean up

# clean up transaction date (to months since December 2021)
datesDf = df.date.str.split('/',expand=True).applymap(lambda x:int(x))
datesDf.columns = ['month', 'day', 'year']
def dateToMonths(year,month):
  return (year-22)*12+month
df.date = datesDf.apply(lambda x: dateToMonths(x.year, x.month), axis=1)


# clean up floor (mean of range provided)
floorsDf = df.floor.str.split(' to ',expand=True).applymap(lambda x:int(x))
df.floor = floorsDf[0] + floorsDf[1]
df.floor = df.floor.apply(lambda x: x/2)

# clean up area (to just sqm)
df.area = df.area.str.split('\n',expand=True)[0].astype(float)

# clean up lease (to years remaining)
leaseDf = df.lease.str.extract('(\d+) years\n?(\d*)')
leaseDf[1] = leaseDf[1].replace('',0)
df.lease = leaseDf[0].astype(float) + leaseDf[1].astype(float)/12

adds = (df.block + df.street).str.replace(u'\xa0', u' ')
latLongs = adds.apply(lambda x: latLong[x])
df['lat'] = latLongs.apply(lambda x: x[0])
df['long'] = latLongs.apply(lambda x: x[1])
df = df.drop(['block', 'street'], axis=1)

print(df)
df.dtypes

     floor  area   top      lease     price  date      lat       long
0      2.0  67.0  1977  53.166667  300500.0    17  1.44154  103.77210
1     11.0  65.0  1976  51.750000  315000.0    17  1.44198  103.77847
2     23.0  67.0  2015  91.250000  430000.0    17  1.43913  103.77814
3      8.0  68.0  2018  94.416667  445000.0    17  1.43834  103.80388
4      8.0  68.0  2018  94.416667  450000.0    17  1.43862  103.80561
..     ...   ...   ...        ...       ...   ...      ...        ...
302    8.0  67.0  2015  92.166667  395000.0     5  1.44780  103.80344
303   14.0  67.0  2015  91.750000  420000.0     5  1.43830  103.79489
304    5.0  71.0  2017  94.666667  420000.0     5  1.44049  103.79836
305    8.0  68.0  2018  95.250000  400000.0     5  1.44636  103.80527
306    5.0  73.0  1990  67.083333  358000.0     5  1.43407  103.77478

[307 rows x 8 columns]
     floor  area   top      lease     price  date      lat       long
0      2.0  67.0  1977  53.166667  300500.0    17  1.44154  103.77

floor    float64
area     float64
top        int64
lease    float64
price    float64
date       int64
lat      float64
long     float64
dtype: object

In [186]:
# master learner

dls = TabularDataLoaders.from_df(df,
                                  y_names='price',
                                  cont_names = ['floor', 'area', 'lease', 'date', 'lat', 'long'],
                                  procs = [Normalize],
                                  valid_idx=valid_idx
)
dls.show_batch()

,floor,area,lease,date,lat,long,price
0,8.0,65.0,52.166667,12.0,1.44198,103.778473,302000.0
1,14.0,65.0,52.416667,9.0,1.44198,103.778473,290000.0
2,5.0,74.0,59.750000,5.0,1.44622,103.773109,265000.0
3,11.0,68.0,94.750000,13.0,1.43916,103.804192,448000.0
4,5.0,67.0,61.250000,14.0,1.43612,103.782059,350000.0
5,5.0,68.0,94.916664,11.0,1.43916,103.804192,440000.0
6,5.0,68.0,94.833336,13.0,1.43916,103.804192,425000.0
7,14.0,68.0,57.916668,5.0,1.44408,103.773697,316000.0
8,2.0,68.0,57.750000,8.0,1.44408,103.773697,310000.0
9,2.0,65.0,50.583332,13.0,1.43995,103.776550,312000.0


In [99]:
# Attempt 1: naive (no extra clean up)

dls = TabularDataLoaders.from_df(df,
                                  y_names='price',
                                  cat_names = ['block', 'street', 'floor', 'area', 'lease', 'date'],
                                  #cont_names = ['top'],
                                  procs = [Normalize, Categorify],
                                  valid_idx=valid_idx
)
dls.show_batch()

,block,street,floor,area,lease,date,top,price
0,694C,Woodlands Dr 62,13 to 15,68.00\nModel A,94 years\n10 months,12/01/22,2018.000000,434000.0
1,129,Marsiling Rise,01 to 03,69.00\nImproved,61 years\n3 months,11/01/22,1985.000000,298000.0
2,573C,Woodlands Dr 16,01 to 03,68.00\nModel A,94 years\n9 months,10/01/22,2018.000000,410000.0
3,24,Marsiling Dr,13 to 15,67.00\nNew Generation,54 years\n1 month,06/01/22,1977.000001,295000.0
4,694D,Woodlands Dr 62,01 to 03,68.00\nModel A,94 years\n6 months,04/01/23,2018.000000,420000.0
5,180B,Marsiling Rd,16 to 18,67.00\nModel A,91 years\n6 months,02/01/23,2015.000000,405000.0
6,784C,Woodlands Rise,04 to 06,68.00\nModel A,95 years\n2 months,06/01/22,2018.000000,410000.0
7,206,Marsiling Dr,04 to 06,68.00\nNew Generation,57 years\n8 months,09/01/22,1981.000000,308888.0
8,111,Woodlands St 13,01 to 03,68.00\nNew Generation,71 years\n6 months,08/01/22,1995.000000,325000.0
9,784C,Woodlands Rise,04 to 06,68.00\nModel A,94 years\n8 months,12/01/22,2018.000000,410000.0


In [ ]:
# Attempt 2a: remove lease

dls = TabularDataLoaders.from_df(df,
                                  y_names='price',
                                  cat_names = ['block', 'street', 'floor', 'area', 'date'],
                                  cont_names = ['top'],
                                  procs = [Normalize, Categorify],
                                  valid_idx=valid_idx
)
dls.show_batch()

In [ ]:
# Attempt 2b: remove top

dls = TabularDataLoaders.from_df(df,
                                  y_names='price',
                                  cat_names = ['block', 'street', 'floor', 'area', 'lease', 'date'],
                                  #cont_names = ['top'],
                                  procs = [Normalize, Categorify],
                                  valid_idx=valid_idx
)
dls.show_batch()

In [111]:
# Attempt 2c.1: clean up lease date into float. use this series instead of lease/top
leaseDf = df.lease.str.extract('(\d+) years\n?(\d*)')
leaseDf[1] = leaseDf[1].replace('',0)
df.lease = leaseDf[0].astype(float) + leaseDf[1].astype(float)/12
print(df)


,block,street,floor,area,date,lease,price
0,574B,Woodlands Dr 16,10 to 12,68.00\nModel A,11/01/22,94.666664,446888.0
1,31,Marsiling Dr,04 to 06,73.00\nNew Generation,10/01/22,53.666667,308000.0
2,314,Woodlands St 31,10 to 12,73.00\nModel A,11/01/22,69.916664,390888.0
3,24,Marsiling Dr,13 to 15,67.00\nNew Generation,06/01/22,54.083333,295000.0
4,887B,Woodlands Dr 50,07 to 09,67.00\nPremium Apartment,11/01/22,91.333336,407000.0
5,148,Woodlands St 13,07 to 09,68.00\nNew Generation,06/01/22,61.833332,328000.0
6,145,Woodlands St 13,07 to 09,68.00\nNew Generation,08/01/22,61.666668,348000.0
7,24,Marsiling Dr,13 to 15,82.00\nNew Generation,05/01/22,54.166667,330000.0
8,302A,Woodlands St 31,25 to 27,67.00\nModel A,12/01/22,92.916664,460000.0
9,157,Woodlands St 13,10 to 12,73.00\nModel A,04/01/23,61.000000,385000.0


In [112]:
# Attempt 2c.2: use lease as cont
dls = TabularDataLoaders.from_df(df,
                                  y_names='price',
                                  cat_names = ['block', 'street', 'floor', 'area', 'date'],
                                  cont_names = ['lease'],
                                  procs = [Normalize, Categorify],
                                  valid_idx=valid_idx
)
dls.show_batch()

,block,street,floor,area,date,lease,price
0,693A,Woodlands Ave 6,01 to 03,68.00\nModel A,01/01/23,94.833336,410000.0
1,574B,Woodlands Dr 16,01 to 03,68.00\nModel A,01/01/23,94.500000,410000.0
2,212,Marsiling Cres,04 to 06,74.00\nModel A,05/01/22,59.750000,265000.0
3,784C,Woodlands Rise,10 to 12,68.00\nModel A,12/01/22,94.750000,430000.0
4,694A,Woodlands Dr 62,04 to 06,68.00\nModel A,11/01/22,94.916664,440000.0
5,15,Marsiling Lane,13 to 15,65.00\nImproved,09/01/22,52.416667,290000.0
6,204,Marsiling Dr,07 to 09,68.00\nNew Generation,01/01/23,57.333332,325000.0
7,693C,Woodlands Ave 6,10 to 12,68.00\nModel A,03/01/23,94.583336,452000.0
8,319,Woodlands St 31,04 to 06,73.00\nModel A,01/01/23,69.333336,380000.0
9,694D,Woodlands Dr 62,04 to 06,68.00\nModel A,03/01/23,94.583336,434000.0


In [124]:
# Attempt 3.1: clean up area
df.area = df.area.str.split('\n',expand=True)[0].astype(float)
print(df)

    block            street     floor  area   top                lease   
0      24      Marsiling Dr  01 to 03  67.0  1977  53 years\n2 months   \
1      15    Marsiling Lane  10 to 12  65.0  1976  51 years\n9 months    
2    180A      Marsiling Rd  22 to 24  67.0  2015  91 years\n3 months    
3    693B   Woodlands Ave 6  07 to 09  68.0  2018  94 years\n5 months    
4    694C   Woodlands Dr 62  07 to 09  68.0  2018  94 years\n5 months    
..    ...               ...       ...   ...   ...                  ...   
302  782C    Woodlands Cres  07 to 09  67.0  2015  92 years\n2 months    
303  886A   Woodlands Dr 50  13 to 15  67.0  2015  91 years\n9 months    
304  717A   Woodlands Dr 70  04 to 06  71.0  2017  94 years\n8 months    
305  785B    Woodlands Rise  07 to 09  68.0  2018  95 years\n3 months    
306   158   Woodlands St 13  04 to 06  73.0  1990   67 years\n1 month    

        price      date  
0    300500.0  05/01/23  
1    315000.0  05/01/23  
2    430000.0  05/01/23  
3    44

In [125]:
# Attempt 3.2: use area as cont
dls = TabularDataLoaders.from_df(df,
                                  y_names='price',
                                  cat_names = ['block', 'street', 'floor', 'lease', 'date'],
                                  cont_names = ['top', 'area'],
                                  procs = [Normalize, Categorify],
                                  valid_idx=valid_idx
)
dls.show_batch()

,block,street,floor,lease,date,top,area,price
0,573C,Woodlands Dr 16,07 to 09,94 years\n9 months,11/01/22,2018.0,68.0,408000.0
1,166,Woodlands St 13,01 to 03,62 years\n6 months,03/01/23,1986.0,67.0,325000.0
2,780C,Woodlands Cres,10 to 12,92 years\n1 month,07/01/22,2015.0,67.0,420000.0
3,180B,Marsiling Rd,28 to 30,92 years\n2 months,06/01/22,2015.0,67.0,440000.0
4,131,Marsiling Rise,04 to 06,61 years\n7 months,07/01/22,1985.0,69.0,320000.0
5,818,Woodlands St 82,01 to 03,70 years\n10 months,03/01/23,1994.0,69.0,360000.0
6,570A,Woodlands Ave 1,01 to 03,89 years\n11 months,01/01/23,2013.0,67.0,395000.0
7,782C,Woodlands Cres,07 to 09,92 years\n2 months,05/01/22,2015.0,67.0,395000.0
8,161,Woodlands St 13,04 to 06,66 years\n5 months,12/01/22,1990.0,73.0,370000.0
9,693B,Woodlands Ave 6,07 to 09,94 years\n5 months,05/01/23,2018.0,68.0,445000.0


In [137]:
# Attempt 4.1: clean up transaction date (to months since December 2021)
datesDf = df.date.str.split('/',expand=True).applymap(lambda x:int(x))
datesDf.columns = ['month', 'day', 'year']
def dateToMonths(year,month):
  return (year-22)*12+month
df.date = datesDf.apply(lambda x: dateToMonths(x.year, x.month), axis=1)
print(df)

In [139]:
# Attempt 4.2: use date as cont
dls = TabularDataLoaders.from_df(df,
                                  y_names='price',
                                  cat_names = ['block', 'street', 'floor', 'lease', 'area'],
                                  cont_names = ['top', 'date'],
                                  procs = [Normalize, Categorify],
                                  valid_idx=valid_idx
)
dls.show_batch()

,block,street,floor,lease,area,top,date,price
0,573C,Woodlands Dr 16,10 to 12,94 years\n4 months,68.00\nModel A,2018.000000,15.0,445500.0
1,19,Marsiling Lane,01 to 03,63 years\n3 months,60.00\nImproved,1987.000001,14.0,318000.0
2,574B,Woodlands Dr 16,13 to 15,94 years\n10 months,68.00\nModel A,2018.000000,9.0,470000.0
3,694A,Woodlands Dr 62,10 to 12,94 years\n9 months,68.00\nModel A,2018.000000,13.0,448000.0
4,24,Marsiling Dr,13 to 15,54 years\n1 month,82.00\nNew Generation,1977.000001,6.0,337000.0
5,570B,Woodlands Ave 1,01 to 03,90 years\n5 months,67.00\nModel A,2013.000000,7.0,365000.0
6,24,Marsiling Dr,10 to 12,54 years\n2 months,67.00\nNew Generation,1977.000001,5.0,310000.0
7,316,Woodlands St 31,04 to 06,69 years\n1 month,73.00\nModel A,1993.000000,15.0,380000.0
8,204,Marsiling Dr,04 to 06,57 years\n1 month,68.00\nNew Generation,1981.000000,16.0,335000.0
9,311,Woodlands St 31,07 to 09,69 years\n7 months,73.00\nModel A,1993.000000,15.0,410000.0


In [165]:
# Attempt 5.1: clean up floor (mean of range provided)
floorsDf = df.floor.str.split(' to ',expand=True).applymap(lambda x:int(x))
df.floor = floorsDf[0] + floorsDf[1]
df.floor = df.floor.apply(lambda x: x/2)
print(df)

    block            street  floor                      area   top   
0      24      Marsiling Dr    2.0     67.00\nNew Generation  1977  \
1      15    Marsiling Lane   11.0           65.00\nImproved  1976   
2    180A      Marsiling Rd   23.0            67.00\nModel A  2015   
3    693B   Woodlands Ave 6    8.0            68.00\nModel A  2018   
4    694C   Woodlands Dr 62    8.0            68.00\nModel A  2018   
..    ...               ...    ...                       ...   ...   
302  782C    Woodlands Cres    8.0            67.00\nModel A  2015   
303  886A   Woodlands Dr 50   14.0  67.00\nPremium Apartment  2015   
304  717A   Woodlands Dr 70    5.0            71.00\nModel A  2017   
305  785B    Woodlands Rise    8.0            68.00\nModel A  2018   
306   158   Woodlands St 13    5.0            73.00\nModel A  1990   

                   lease     price      date  
0    53 years\n2 months   300500.0  05/01/23  
1    51 years\n9 months   315000.0  05/01/23  
2    91 years\n3 m

In [166]:
# Attempt 5.2: use floor as cont
dls = TabularDataLoaders.from_df(df,
                                  y_names='price',
                                  cat_names = ['block', 'street', 'date', 'lease', 'area'],
                                  cont_names = ['top', 'floor'],
                                  procs = [Normalize, Categorify],
                                  valid_idx=valid_idx
)
dls.show_batch()

,block,street,date,lease,area,top,floor,price
0,151,Woodlands St 13,11/01/22,61 years\n5 months,73.00\nModel A,1985.000000,5.0,352000.0
1,132,Marsiling Rise,09/01/22,71 years\n8 months,82.00\nNew Generation,1995.000000,5.0,350000.0
2,319,Woodlands St 31,01/01/23,69 years\n4 months,73.00\nModel A,1993.000000,5.0,380000.0
3,574B,Woodlands Dr 16,11/01/22,94 years\n8 months,68.00\nModel A,2018.000000,11.0,446888.0
4,784B,Woodlands Rise,12/01/22,94 years\n9 months,68.00\nModel A,2018.000000,14.0,445000.0
5,2,Marsiling Dr,03/01/23,50 years\n4 months,65.00\nImproved,1973.999999,5.0,315000.0
6,694C,Woodlands Dr 62,04/01/23,94 years\n5 months,68.00\nModel A,2018.000000,2.0,415000.0
7,157,Woodlands St 13,02/01/23,61 years\n1 month,73.00\nModel A,1985.000000,5.0,368000.0
8,15,Marsiling Lane,12/01/22,52 years\n2 months,65.00\nImproved,1976.000000,8.0,302000.0
9,158,Woodlands St 13,10/01/22,66 years\n8 months,73.00\nModel A,1990.000000,5.0,370000.0


In [7]:
# Attempt 6.1: convert block and street to lat/long
adds = (df.block + df.street).str.replace(u'\xa0', u' ')
latLongs = adds.apply(lambda x: latLong[x])
df['lat'] = latLongs.apply(lambda x: x[0])
df['long'] = latLongs.apply(lambda x: x[1])
df = df.drop(['block', 'street'], axis=1)
print(df)

     floor  area  year                lease     price      date      lat   
0      2.0  67.0  1977  53 years\n2 months   300500.0  738641.0  1.44154  \
1     11.0  65.0  1976  51 years\n9 months   315000.0  738641.0  1.44198   
2     23.0  67.0  2015  91 years\n3 months   430000.0  738641.0  1.43913   
3      8.0  68.0  2018  94 years\n5 months   445000.0  738641.0  1.43834   
4      8.0  68.0  2018  94 years\n5 months   450000.0  738641.0  1.43862   
..     ...   ...   ...                  ...       ...       ...      ...   
302    8.0  67.0  2015  92 years\n2 months   395000.0  738276.0  1.44780   
303   14.0  67.0  2015  91 years\n9 months   420000.0  738276.0  1.43830   
304    5.0  71.0  2017  94 years\n8 months   420000.0  738276.0  1.44049   
305    8.0  68.0  2018  95 years\n3 months   400000.0  738276.0  1.44636   
306    5.0  73.0  1990   67 years\n1 month   358000.0  738276.0  1.43407   

          long  
0    103.77210  
1    103.77847  
2    103.77814  
3    103.80388  
4 

In [8]:
# Attempt 6.2: use lat/long as cont
#dls = TabularDataLoaders.from_df(df,
#                                  y_names='price',
#                                  cat_names = ['date', 'floor', 'lease', 'area'],
#                                  cont_names = ['top', 'lat', 'long'],
#                                  procs = [Normalize, Categorify],
#                                  valid_idx=valid_idx
#)
dls = TabularDataLoaders.from_df(df,
                                  y_names='price',
                                  cont_names = ['year', 'lat', 'long', 'floor', 'date', 'area'],
                                  procs = [Normalize],
                                  valid_idx=range(60)
)
dls.show_batch()

,year,lat,long,floor,date,area,price
0,1974.000000,1.43995,103.776550,8.0,738307.000003,65.0,300000.0
1,1976.000000,1.44109,103.779251,11.0,738399.000000,76.0,330000.0
2,1984.000000,1.44273,103.778038,5.0,738337.000001,65.0,305000.0
3,2018.000000,1.44565,103.804993,14.0,738490.000002,68.0,445000.0
4,1977.000001,1.44151,103.773300,5.0,738399.000000,67.0,305000.0
5,1985.000000,1.43437,103.774803,11.0,738460.000000,73.0,350000.0
6,2015.000000,1.43913,103.778351,17.0,738551.999997,67.0,405000.0
7,1981.000000,1.44537,103.772926,5.0,738399.000000,68.0,308888.0
8,1985.000000,1.43611,103.781174,11.0,738490.000002,67.0,358000.0
9,2018.000000,1.42970,103.796852,2.0,738367.999998,68.0,415000.0


In [14]:
learn = tabular_learner(dls, metrics=[R2Score(), rmse], y_range=(200000,550000))
learn.fit_one_cycle(40)

epoch,train_loss,valid_loss,r2_score,_rmse,time
0,8295102976.000000,2791767552.000000,-0.039412,52837.179688,00:00
1,7826882048.000000,2671344640.000000,0.005423,51685.050781,00:00
2,7068063232.000000,2248840192.000000,0.162727,47421.937500,00:00
3,5921096704.000000,1478842624.000000,0.449407,38455.722656,00:00
4,4875693056.000000,734417792.000000,0.726566,27100.144531,00:00
5,4260475904.000000,442338848.000000,0.835311,21031.853516,00:00
6,3868805376.000000,474764960.000000,0.823239,21789.101562,00:00
7,3535457536.000000,517416544.000000,0.807359,22746.791016,00:00
8,3176715264.000000,564213376.000000,0.789936,23753.175781,00:00
9,2833245952.000000,792446144.000000,0.704962,28150.419922,00:00


In [15]:
learn.show_results()

,year,lat,long,floor,date,area,price,price_pred
0,-0.283514,-1.325520,-0.761924,-0.515258,1.799361,1.122544,380000.0,383722.43750
1,-1.192229,0.553416,-1.013810,-1.136080,1.799361,-0.374854,308000.0,308619.78125
2,-0.737871,-0.785449,-0.890373,0.726388,2.161864,1.122544,385000.0,385937.00000
3,-0.226719,0.583415,0.372190,-1.136080,1.799361,0.124279,360000.0,386333.00000
4,0.965970,0.107120,-0.616557,1.968032,2.161864,-0.624420,422000.0,407698.90625
5,1.136354,-0.117895,1.513196,-0.515258,1.799361,-0.125288,430000.0,433002.56250
6,-0.965050,1.277236,-1.044512,-1.136080,2.161864,-0.125288,335000.0,313720.18750
7,-0.737871,-0.521047,-1.035740,-0.515258,2.161864,-0.125288,352000.0,352935.28125
8,-0.453898,-0.909223,-0.907917,0.105565,1.799361,1.122544,390000.0,385760.90625


In [40]:
# predict
#testData = pd.DataFrame([[2, 67, 53.1667, 17, 1.40416, 103.78597]], columns=['floor', 'area', 'lease', 'date', 'lat', 'long'])
#testData
#learn.predict(df.iloc[0])
#learn.predict(testData.iloc[0])
row, clas, prob = learn.predict(df.iloc[0].drop('price'))
df
print(row)

       year       lat      long    floor      date      area        price
0 -1.192229  0.559048 -1.112377 -1.13608  2.512674 -0.374854  317598.5625
